# Rewriting TransferFuncList

In [2]:
%load_ext autoreload
import sys
sys.path.append("../..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]

# SET THIS TO TRUE TO USE LATEX
matplotlib.rcParams['text.usetex'] = False

from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.low_energy.lowE_deposition import compute_fs
import main

from tqdm import tqdm_notebook as tqdm

np.set_printoptions(threshold=np.nan)

In [7]:
standard_tfs = False
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_fixed_He/'

# True if you would like to subtract the CMB off of the transfer functions?
CMB_subtracted=True

if not standard_tfs:
    direc_arr = [direc, direc]
    xes = np.array([
        0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.8, -1.4, -1, -.8, -0.65, -0.5, -.24, 0, .2, 0.4, .7, 1.3, 2.2, 3.1, 4]),
        None]
    )
    string_arr = ["_xHe0", "_standard_xe_xHe_full_rs"]
    switching_rs = 1600
else:
    direc_arr = direc+'tfs_standard_xe_xHe_full_rs/'
    xes = None
    string_arr = ["_standard_xe_xHe_full_rs"]
    
string_arr = ["_xHe0"]
    
for ii, string in enumerate(string_arr):
    print('Loading transfer functions...')
    highengphot_tflist_arr = pickle.load(open(direc_arr[ii]+"tfunclist_photspec_60eV_complete"+string+".raw", "rb"))
    print('Loaded high energy photons...')

    lowengphot_tflist_arr  = pickle.load(open(direc_arr[ii]+"tfunclist_lowengphotspec_60eV_complete"+string+".raw", "rb"))
    print('Low energy photons...')

    lowengelec_tflist_arr  = pickle.load(open(direc_arr[ii]+"tfunclist_lowengelecspec_60eV_complete"+string+".raw", "rb"))
    print('Low energy electrons...')

    highengdep_arr = pickle.load(open(direc_arr[ii]+"highdeposited_60eV_complete"+string+".raw", "rb"))
    highengdep_arr = np.swapaxes(highengdep_arr, 1, 2)
    print('high energy deposition.\n')

    CMB_engloss_arr = pickle.load(open(direc_arr[ii]+"CMB_engloss_60eV_complete"+string+".raw", "rb"))
    CMB_engloss_arr = np.swapaxes(CMB_engloss_arr, 1, 2)
    print('CMB losses.\n')

Loading transfer functions...
Loaded high energy photons...
Low energy photons...
Low energy electrons...
high energy deposition.

CMB losses.



In [11]:
len(highengphot_tflist_arr)

19